## Graph Classification

In [1]:
#  dataset from https://snap.stanford.edu/data/deezer_ego_nets.html
import json
import numpy as np
import pandas as pd

import torch
from torch.nn import Linear
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

/Users/tabaneslami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
with open('deezer_ego_nets/deezer_edges.json') as f:
    graphs = json.load(f)


In [3]:
graph_labels = pd.read_csv('deezer_ego_nets/deezer_target.csv')
graph_labels = graph_labels['target'].to_numpy()

In [4]:
def creat_graph(edge_list, label):
    '''
    creates torch geometric graph from 
    edge list and target
    '''
    edge_list = torch.tensor(np.array(edge_list).T)
    label = torch.tensor(label)
    feature_vector = torch.ones((torch.max(edge_list).item()+1),1)
    graph = Data(x=feature_vector, edge_index = edge_list, label = label)
    return graph

In [5]:
all_graphs = []
for i, key in enumerate(graphs.keys()):
    all_graphs.append(creat_graph(graphs[str(key)], graph_labels[i]))
    
train_loader = DataLoader(all_graphs, batch_size=2, shuffle=True)


In [6]:
class graph_classifier(torch.nn.Module):
    def __init__(self, num_features, hidden_channel_1, hidden_channnel2, num_class):
        super(graph_classifier, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channel_1)
        self.conv2 = GCNConv(hidden_channel_1, hidden_channnel2)
        self.linear = Linear(hidden_channnel2, num_class)
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = self.linear(x)
        return x

In [8]:
model = graph_classifier(1, 20, 10, 2)


In [10]:
for i, data in enumerate(train_loader):
    out = model(data.x, data.edge_index, data.batch)